In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

all_data = pd.read_csv('sales所在周次节假日_剔除换货.csv')
all_data.rename(columns={'店铺名称':'store','地理位置指数':'position','年份':'years','周次':'weeks of year','是否包含法定节假日':'Isholiday','季节':'season','大类':'item','销量(剔除换货）':'sales','均价（剔除换货）':'prices','促销（剔除换货）':'discount'},inplace=True)
all_data['weeks'] = all_data['weeks of year']
a = all_data
week_5 = a.loc[lambda a : a['years'] == 2015]
week_6 = a.loc[lambda a : a['years'] == 2016]
week_6['weeks'] += 53
a = pd.concat([week_5,week_6])

from sklearn import preprocessing

#map店铺名称
name = a['store']
le = preprocessing.LabelEncoder()
le.fit(name)
list(name)
le.transform(name)
tmp = le.transform(name)
a['store'] = tmp

#map合并大类
category = a['item']
le.fit(category)
list(category)
le.transform(category)
tmp = le.transform(category)
a['item'] = tmp

#map季节
season = a['season']
le.fit(season)
list(season)
le.transform(season)
tmp = le.transform(season)
a['season'] = tmp


/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [2]:
df = a
masked_series = (df['years']==2016) & (df['weeks'].isin([53,54,55]))
masked_series2 = (df['years']==2016) & (~(df['weeks'].isin([53,54,55])))
df.loc[(masked_series), 'train_or_test'] = 'test'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df['years']==2015,:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))


Train shape: (8119, 12)
Test shape: (340, 12)


In [3]:
df_n = df.drop(['train_or_test'],axis=1)

#log sales,prices
df_n['sales'] = np.log1p(df.sales.values)
df_n['prices'] = np.log1p(df.sales.values)
df_n.sample(2)

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
15184,25,7,2015,32,0,1,3,1.791759,1.791759,0.366042,32
17071,28,7,2015,46,0,0,2,2.484907,2.484907,0.311867,46


In [4]:
train = df_n[lambda df_n: df_n['years']==2015]
t1 = df_n[lambda df_n: df_n['weeks']==54]
t2 = df_n[lambda df_n: df_n['weeks']==55]
t3 = df_n[lambda df_n: df_n['weeks']==56]
test = pd.concat([t1,t2,t3])

y_train = train['sales']
y_test = test['sales']
X_train = train.drop('sales', axis=1)
X_test = test.drop('sales', axis=1)

In [1]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb